## Spark setting

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [3]:
import findspark
findspark.init("spark-3.0.3-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

## Kaggle APIs

________________________________________________________________________

In [5]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"federicofiorio","key":"3c9fda0dd26cc20d5f652a577743142c"}'}

In [6]:
! pip install kaggle

In [7]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
!cp kaggle.json ~/.kaggle/

In [9]:
!ls ~/.kaggle

kaggle.json


In [10]:
!chmod 600 /root/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
!ls

 0401_UkraineCombinedTweetsDeduped.csv.gzip
 0402_UkraineCombinedTweetsDeduped.csv.gzip
 0403_UkraineCombinedTweetsDeduped.csv.gzip
 0404_UkraineCombinedTweetsDeduped.csv.gzip
 0405_UkraineCombinedTweetsDeduped.csv.gzip
 0406_UkraineCombinedTweetsDeduped.csv.gzip
 0407_UkraineCombinedTweetsDeduped.csv.gzip
 0408_UkraineCombinedTweetsDeduped.csv.gzip
 0409_UkraineCombinedTweetsDeduped.csv.gzip
 0410_UkraineCombinedTweetsDeduped.csv.gzip
 0411_UkraineCombinedTweetsDeduped.csv.gzip
 0412_UkraineCombinedTweetsDeduped.csv.gzip
 0413_UkraineCombinedTweetsDeduped.csv.gzip
 0414_UkraineCombinedTweetsDeduped.csv.gzip
 0415_UkraineCombinedTweetsDeduped.csv.gzip
 0416_UkraineCombinedTweetsDeduped.csv.gzip
 0417_UkraineCombinedTweetsDeduped.csv.gzip
 0418_UkraineCombinedTweetsDeduped.csv.gzip
 0419_UkraineCombinedTweetsDeduped.csv.gzip
 0420_UkraineCombinedTweetsDeduped.csv.gzip
 0421_UkraineCombinedTweetsDeduped.csv.gzip
 0422_UkraineCombinedTweetsDeduped.csv.gzip
 0423_UkraineCombinedTweetsDedup

In [13]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
replace 0401_UkraineCombinedTweetsDeduped.csv.gzip? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Use the data

________________________________________________________________________

In [14]:
import numpy as np
import pandas as pd
import csv
import os
import warnings

In [15]:
filename = r"0401_UkraineCombinedTweetsDeduped.csv.gzip"
df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
df[0:2].T

,0,1
userid,16882774,3205296069
username,Yaniela,gregffff
acctdesc,"Animal lover, supports those who fight injusti...",NaN
location,Hawaii,NaN
following,1158,122
followers,392,881
totaltweets,88366,99853
usercreatedts,2008-10-21 07:34:04.000000,2015-04-25 11:24:34.000000
tweetid,1509681950042198030,1509681950151348229
tweetcreatedts,2022-04-01 00:00:00.000000,2022-04-01 00:00:00.000000


In [17]:
type(df)

pandas.core.frame.DataFrame

In [18]:
df = df[['tweetid', 'text', 'hashtags','language']] #keep only 3 columns
df.head()

,tweetid,text,hashtags,language
0,1509681950042198030,⚡The Ukrainian Air Force would like to address...,[],en
1,1509681950151348229,Chernihiv oblast. Ukrainians welcome their lib...,"[{'text': 'russianinvasion', 'indices': [77, 9...",en
2,1509681950683926556,America 🇺🇸 is preparing for something worse th...,"[{'text': 'RussianUkrainianWar', 'indices': [7...",en
3,1509681951116046336,JUST IN: #Anonymous has hacked &amp; released ...,"[{'text': 'Anonymous', 'indices': [25, 35]}]",en
4,1509681951304990720,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,[],en


In [19]:
pd.options.display.max_colwidth = 1200
print(df[df['tweetid']==	1509681950042198030]["text"],df[df['tweetid']==	1509681950042198030]["hashtags"])

0    ⚡The Ukrainian Air Force would like to address misinformation published in multiple Western media outlets regarding the situation in the 🇺🇦 sky and support from our @NATO allies. \nMore in 🧵(1/16)\n#ProtectUАSky #StopRussia #UkraineUnderAttaсk
Name: text, dtype: object 0    []
Name: hashtags, dtype: object


As we can see in the previous cell, the tweet has the hashtag #StopRussia #UkraineUnderAttaсk BUT the dataframe (1st tuple) sais there are no hashtag in the tweet. We will try to recover form this.

In [20]:
from pyspark.sql.types import *

schema = StructType([StructField("tweetid", StringType(), True)\
                   ,StructField("text", StringType(), True)\
                   ,StructField("hashtags", StringType(), True)\
                   ,StructField("language", StringType(), True)])

#create spark dataframe using schema
df_spark = spark.createDataFrame(df,schema=schema)

In [21]:
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [22]:
df_spark.show()

+-------------------+--------------------+--------------------+--------+
|            tweetid|                text|            hashtags|language|
+-------------------+--------------------+--------------------+--------+
|1509681950042198030|⚡The Ukrainian Ai...|                  []|      en|
|1509681950151348229|Chernihiv oblast....|[{'text': 'russia...|      en|
|1509681950683926556|America 🇺🇸 is p...|[{'text': 'Russia...|      en|
|1509681951116046336|JUST IN: #Anonymo...|[{'text': 'Anonym...|      en|
|1509681951304990720|***PUBLIC MINT NO...|                  []|      en|
|1509681952000937999|The Amazing story...|[{'text': 'Russia...|      en|
|1509681952978210849|&amp;quot;How we ...|                  []|      en|
|1509681953053843466|India's purchase ...|[{'text': 'Russia...|      en|
|1509681953091457035|The most basic te...|[{'text': 'Ukrain...|      en|
|1509681953418711050|"The image that R...|[{'text': 'Putin'...|      en|
|1509681953418752008|#Russia’s Preside...|[{'text': '

## RDD TEXT SERVE PER PRENDERE GLI HASHTAG DAL TWEET IN QUANTO LA COLONNA HASHTAG DEL DATASET NON è ACCURATA

In [23]:
import nltk
from nltk import word_tokenize
import re
nltk.download('punkt') #to make it work

rdd = df_spark.rdd #to get the rdd from dataframe

rdd.take(10)
rdd=rdd.filter(lambda x: x[3] == "en") #filter out non english tweets


rdd_text = rdd.map(lambda x : (x[0], x[1])).map(lambda x:(x[0], word_tokenize(x[1]))) #id, word_tokenized

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## ESTRAGGO SOLO GLI HASHTAGS

In [24]:
def processHashtags(tweet):
  hashtags = []
  for i,word in enumerate(tweet):
    if word == '#':
      try:
        hashtags.append(re.sub(r'[^\w\s]','',tweet[i+1]).lower()) #remove punctuation with regexp and put them lowercase
      except:
        pass
  return hashtags

hashtags_per_tweet = rdd_text.map(lambda x: (x[0], processHashtags(x[1])))
hashtags_per_tweet.first()

('1509681950042198030', ['protectuаsky', 'stoprussia', 'ukraineunderattaсk'])

In [25]:
#reconvert the rdd to spark dataframe
deptColumns = ["tweet","hashtag"]
new_df_spark = hashtags_per_tweet.toDF(deptColumns)
new_df_spark.printSchema()
new_df_spark.show(truncate=False)

root
 |-- tweet: string (nullable = true)
 |-- hashtag: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tweet              |hashtag                                                                                                                                                                   |
+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1509681950042198030|[protectuаsky, stoprussia, ukraineunderattaсk]                                                                                                                            |
|1509681950151348229|[russianinvasion, standwithukraine, ukraineunderattack, ukrainewillwin, putini

APRIORI

In [26]:
hashtag=new_df_spark.select('hashtag')

In [27]:
basket_file = hashtag.rdd.flatMap(list)

In [28]:
basket_file.take(5)

[['protectuаsky', 'stoprussia', 'ukraineunderattaсk'],
 ['russianinvasion',
  'standwithukraine',
  'ukraineunderattack',
  'ukrainewillwin',
  'putinisawarcriminal',
  'stopputin',
  'russianukrainianwar',
  'russiagohome',
  'россиясмотри',
  'нетвойне'],
 ['russianukrainianwar', 'china', 'taiwan'],
 ['anonymous', 'oprussia', 'ddosecrets'],
 ['nft', 'mint']]

## Riduco dimensioni baskets per essere veloci

In [29]:
basket_file = basket_file.take(5000)

In [30]:
basket_file = spark.sparkContext.parallelize(basket_file)

In [31]:
type(basket_file)

pyspark.rdd.RDD

In [32]:
basket_file.take(10)

[['protectuаsky', 'stoprussia', 'ukraineunderattaсk'],
 ['russianinvasion',
  'standwithukraine',
  'ukraineunderattack',
  'ukrainewillwin',
  'putinisawarcriminal',
  'stopputin',
  'russianukrainianwar',
  'russiagohome',
  'россиясмотри',
  'нетвойне'],
 ['russianukrainianwar', 'china', 'taiwan'],
 ['anonymous', 'oprussia', 'ddosecrets'],
 ['nft', 'mint'],
 ['russia',
  'ukraine',
  'motivation',
  'netde',
  'edude',
  'delaware',
  'government',
  'usa'],
 ['ukraine', 'ukrainewar', 'russia', 'ukraineinvasion'],
 ['russian', 'moscow'],
 ['ukraine'],
 ['putin', 'medvedev', 'russia', 'ukraine']]

_______________________________________________________________________________

In [47]:
#DA RIVEDERE
count = 250000 #obtaine doing count=basket_file.count() (too much time to repeat this all the time)
threshold = 1
print(count, threshold)

250000 1


Start apriori

In [48]:
#setp 1 calculate the freq of each item in the basket file
singleton=basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)

singleton.take(20)

[('ukraineunderattaсk', 371),
 ('russianinvasion', 106),
 ('standwithukraine', 494),
 ('ukrainewillwin', 142),
 ('stopputin', 157),
 ('russianukrainianwar', 196),
 ('russiagohome', 117),
 ('нетвойне', 125),
 ('taiwan', 8),
 ('anonymous', 132),
 ('ddosecrets', 20),
 ('netde', 1),
 ('delaware', 1),
 ('usa', 63),
 ('ukraineinvasion', 18),
 ('moscow', 12),
 ('putin', 439),
 ('medvedev', 1),
 ('russianarmy', 40),
 ('soviet', 15)]

In [49]:
#step 2: filter out all the non frequent singleton
freq_singleton=singleton.filter(lambda x: x[1]>=threshold)

freq_singleton.take(20)

[('ukraineunderattaсk', 371),
 ('russianinvasion', 106),
 ('standwithukraine', 494),
 ('ukrainewillwin', 142),
 ('stopputin', 157),
 ('russianukrainianwar', 196),
 ('russiagohome', 117),
 ('нетвойне', 125),
 ('taiwan', 8),
 ('anonymous', 132),
 ('ddosecrets', 20),
 ('netde', 1),
 ('delaware', 1),
 ('usa', 63),
 ('ukraineinvasion', 18),
 ('moscow', 12),
 ('putin', 439),
 ('medvedev', 1),
 ('russianarmy', 40),
 ('soviet', 15)]

In [50]:
#step 3: now create all the possible pairs made up by frequen singletons

from itertools import combinations
pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #without toLocalIterator error (reference https://stackoverflow.com/questions/32771737/convert-an-rdd-to-iterable-pyspark)

pairs[:10]

[('ukraineunderattaсk', 'russianinvasion'),
 ('ukraineunderattaсk', 'standwithukraine'),
 ('ukraineunderattaсk', 'ukrainewillwin'),
 ('ukraineunderattaсk', 'stopputin'),
 ('ukraineunderattaсk', 'russianukrainianwar'),
 ('ukraineunderattaсk', 'russiagohome'),
 ('ukraineunderattaсk', 'нетвойне'),
 ('ukraineunderattaсk', 'taiwan'),
 ('ukraineunderattaсk', 'anonymous'),
 ('ukraineunderattaсk', 'ddosecrets')]

In [51]:
pairs

[('ukraineunderattaсk', 'russianinvasion'),
 ('ukraineunderattaсk', 'standwithukraine'),
 ('ukraineunderattaсk', 'ukrainewillwin'),
 ('ukraineunderattaсk', 'stopputin'),
 ('ukraineunderattaсk', 'russianukrainianwar'),
 ('ukraineunderattaсk', 'russiagohome'),
 ('ukraineunderattaсk', 'нетвойне'),
 ('ukraineunderattaсk', 'taiwan'),
 ('ukraineunderattaсk', 'anonymous'),
 ('ukraineunderattaсk', 'ddosecrets'),
 ('ukraineunderattaсk', 'netde'),
 ('ukraineunderattaсk', 'delaware'),
 ('ukraineunderattaсk', 'usa'),
 ('ukraineunderattaсk', 'ukraineinvasion'),
 ('ukraineunderattaсk', 'moscow'),
 ('ukraineunderattaсk', 'putin'),
 ('ukraineunderattaсk', 'medvedev'),
 ('ukraineunderattaсk', 'russianarmy'),
 ('ukraineunderattaсk', 'soviet'),
 ('ukraineunderattaсk', 'ukrainian'),
 ('ukraineunderattaсk', 'kharkiv'),
 ('ukraineunderattaсk', '30march'),
 ('ukraineunderattaсk', 'peace'),
 ('ukraineunderattaсk', 'zelensky'),
 ('ukraineunderattaсk', 'community'),
 ('ukraineunderattaсk', 'news'),
 ('ukraineun

In [52]:
possible_pairs=basket_file.map(lambda x: list(combinations(x,2)))  #per ogni basket, creo ogni possiblie coppia
pairs_freq_sing=possible_pairs.filter(lambda x: x in pairs)
#ERRORE: QUA CREO TUTTE LE COPPIE POSSIBILI, MA è SBAGLIAT

#flat and count and reduce
flat_pairs=pairs_freq_sing.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)

In [54]:
#step 5: finally filetr out the pairs < thresold
freq_pairs=flat_pairs.filter(lambda x: x[1]>=threshold)

In [ ]:
freq_pairs.take(10)

In [41]:
#step 4 : filter out the non frequent pairs

#flatmap so I'm able to compute reducebyKey, because otherwise I will have different tuples in a list (basket)

#secondo me può  essere ottimizzato perchè il subset è computato su tutto i possibili basket del dataset, quindi subset non serve a niente visto che è perforza un subset di tutti i baskets?
'''count_pairs = basket_file.flatMap(lambda x: ([(tuple(pair), 1) for pair in pairs if set(list(pair)).issubset(set(x))])) \
                              .reduceByKey(lambda a, b: a+b) \
                              .filter(lambda value: value >= threshold)

count_pairs.take(10)'''
count_pairs = basket_file.flatMap(lambda x: ([(pair, 1) for pair in pairs if set(list(pair))]))
count_pairs.take(5)
#count_pairs.take(5)

[(('ukraineunderattaсk', 'russianinvasion'), 1),
 (('ukraineunderattaсk', 'standwithukraine'), 1),
 (('ukraineunderattaсk', 'ukrainewillwin'), 1),
 (('ukraineunderattaсk', 'stopputin'), 1),
 (('ukraineunderattaсk', 'russianukrainianwar'), 1)]

In [42]:
[(('ukraineunderattaсk', 'protectuаsky'), 1),
 (('ukraineunderattaсk', 'stoprussia'), 1),
 (('protectuаsky', 'stoprussia'), 1),
 (('russianinvasion', 'standwithukraine'), 1),
 (('russianinvasion', 'ukrainewillwin'), 1)]

[(('ukraineunderattaсk', 'protectuаsky'), 1),
 (('ukraineunderattaсk', 'stoprussia'), 1),
 (('protectuаsky', 'stoprussia'), 1),
 (('russianinvasion', 'standwithukraine'), 1),
 (('russianinvasion', 'ukrainewillwin'), 1)]

In [43]:
#collect the result
import time
startTime = time.time()
freq_pairs.collect()
print(time.time()-startTime)

0.15861272811889648


In [44]:
pairs_freq_sing=basket_file.map(lambda x: list(combinations(x,2)))
pairs_freq_sing.take(1)

[[('protectuаsky', 'stoprussia'),
  ('protectuаsky', 'ukraineunderattaсk'),
  ('stoprussia', 'ukraineunderattaсk')]]

In [45]:
#filter out the pairs non made by freq singletons
new=pairs_freq_sing.filter(lambda x: x in pairs)
new.first()

ValueError: ignored

In [ ]:
singleton.first()

In [ ]:
while True:
  pass